In [131]:
import re
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#package import
train=pd.read_csv('/content/Data/twitter_training.csv',names=["Tweet_ID", "Entity","Sentiment", "Tweet_Content"])
test=pd.read_csv('/content/Data/twitter_validation.csv',names=["Tweet_ID", "Entity", "Sentiment", "Tweet_Content"])
#data import
#refining the data
train=train.dropna()
#function to clean data
def cleannuptext(txt):
  #delete 'pattern'
  alltweets=txt.append(None,ignore_index = True)
  alltweets['tidy_tweet']=np.vectorize(remove_unwanted)(alltweets['Tweet_Content'],'@[\w]*')
  #remove usernames
  alltweets['tidy_tweet'] = alltweets['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
  #remove punctuation etc
  alltweets['tidy_tweet'] = alltweets['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split()if len(w) > 3]))
  #remove short words
  #alltweets.head()
  tokenized_tweet = alltweets['tidy_tweet'].apply(lambda x: x.split())
  tokenized_tweet.head()
  #tokenize the words
  lemma = nltk.stem.porter.PorterStemmer()
  tokenized_tweet = tokenized_tweet.apply(lambda x:[lemma.stem(i) for i in x])
  tokenized_tweet.head()
  #lemmatizing
  for i in range(len(tokenized_tweet)):
      tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
  alltweets['tidy_tweet'] = tokenized_tweet
  alltweets.head()
  return alltweets
def remove_unwanted(txt,pattern):
    matches = re.findall(pattern,txt)
    for match in matches:
      txt = re.sub(match,'',txt)
    return txt
def toknizetwt(txt):
  labels = np.array(txt['Sentiment'])
  y = []
  for i in range(len(labels)):
    if labels[i] == 'Neutral':
      y.append(0)
    if labels[i] == 'Negative':
      y.append(1)
    if labels[i] == 'Positive':
      y.append(2)
    if labels[i] == 'Irrelevant':
      y.append(-1)
  y = np.array(y)
  return y
train_data=cleannuptext(train)
test_data=cleannuptext(test)
train_data.drop(labels=['Tweet_Content','Tweet_ID','Entity'],axis=1,inplace=True)
test_data.drop(labels=['Tweet_ID','Entity','Tweet_Content'],axis=1,inplace=True)
train_data['Sentiment']=toknizetwt(train_data)
test_data['Sentiment']=toknizetwt(test_data)
#final test and train dataset
train_data.to_csv('training_dataset.csv', index=False)
test_data.to_csv('testing_dataset.csv', index=False)